dataset --> https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019

### Tratamento dos dados

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report,f1_score,accuracy_score, recall_score
from sklearn import metrics
import pickle
from sklearn import tree
import graphviz
import warnings
warnings.filterwarnings("ignore")

In [6]:
tophits_df = pd.read_csv('songs_normalize.csv', sep=',')
tophits_df

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Jonas Brothers,Sucker,181026,False,2019,79,0.842,0.734,1,-5.065,0,0.0588,0.0427,0.000000,0.1060,0.952,137.958,pop
1996,Taylor Swift,Cruel Summer,178426,False,2019,78,0.552,0.702,9,-5.707,1,0.1570,0.1170,0.000021,0.1050,0.564,169.994,pop
1997,Blanco Brown,The Git Up,200593,False,2019,69,0.847,0.678,9,-8.635,1,0.1090,0.0669,0.000000,0.2740,0.811,97.984,"hip hop, country"
1998,Sam Smith,Dancing With A Stranger (with Normani),171029,False,2019,75,0.741,0.520,8,-7.513,1,0.0656,0.4500,0.000002,0.2220,0.347,102.998,pop


In [7]:
#transformando os dados da coluna 'duration_ms' de milisegundo para segundo (divdir por mil)
segundos = []
for s in tophits_df['duration_ms']:
    seg = (s/1000)
    segundos.append(seg)
tophits_df['duration_seg'] = segundos
tophits_df = tophits_df.drop(['duration_ms'], axis=1)

In [8]:
#transformadno a coluna 'explicit' --> False: 0, True: 1
tophits_df['explicit'] = tophits_df['explicit'].apply(lambda x:1 if x == True else 0)

In [9]:
X = tophits_df.drop(['artist', 'song', 'key', 'mode', 'tempo', 'genre', 'year','popularity'], axis=1)
y = tophits_df['popularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [10]:
tophits_df

,artist,song,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,duration_seg
0,Britney Spears,Oops!...I Did It Again,0,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop,211.160
1,blink-182,All The Small Things,0,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop",167.066
2,Faith Hill,Breathe,0,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country",250.546
3,Bon Jovi,It's My Life,0,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal",224.493
4,*NSYNC,Bye Bye Bye,0,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop,200.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Jonas Brothers,Sucker,0,2019,79,0.842,0.734,1,-5.065,0,0.0588,0.0427,0.000000,0.1060,0.952,137.958,pop,181.026
1996,Taylor Swift,Cruel Summer,0,2019,78,0.552,0.702,9,-5.707,1,0.1570,0.1170,0.000021,0.1050,0.564,169.994,pop,178.426
1997,Blanco Brown,The Git Up,0,2019,69,0.847,0.678,9,-8.635,1,0.1090,0.0669,0.000000,0.2740,0.811,97.984,"hip hop, country",200.593
1998,Sam Smith,Dancing With A Stranger (with Normani),0,2019,75,0.741,0.520,8,-7.513,1,0.0656,0.4500,0.000002,0.2220,0.347,102.998,pop,171.029


In [11]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1400, 10) (600, 10) (1400,) (600,)


In [12]:
arvore = DecisionTreeRegressor()
arvore = arvore.fit(X_train, y_train)
arvore

DecisionTreeRegressor()

In [13]:
pickle.dump(arvore, open('arvore.pkl', 'wb'))
pickled_arvore = pickle.load(open('arvore.pkl', 'rb'))
pickled_arvore = pickled_arvore.predict(X_test)

In [14]:
print(f"r2_score: {metrics.r2_score(y_test,pickled_arvore)}")
print(f'mean_squared_error: {np.sqrt(mean_squared_error(y_test, pickled_arvore))}')
print(f"f1_score: {metrics.f1_score(y_test,pickled_arvore, average='macro')}")
print(f"recall_score: {metrics.recall_score(y_test,pickled_arvore,average='macro')}")
print(f"accuracy_score: {metrics.accuracy_score(y_test,pickled_arvore)}")


r2_score: -1.1014546975003796
mean_squared_error: 31.61987771429021
f1_score: 0.05897803073708572
recall_score: 0.076445598847577
accuracy_score: 0.07333333333333333


In [15]:
#Média da popularidade das musícas
tophits_df['popularity'].mean()

59.8725

In [16]:
#Exemplo - Prever se a musica tem altas ou baixas chances de ser popular
variaveis_musica = [0,0.872,0.444,-15.345,0.693,0.23,0.001678,0.0901,0.609,300.289]
variaveis_musica = np.array(variaveis_musica)
variaveis_musica = arvore.predict([variaveis_musica])
variaveis_musica

array([59.])

In [17]:
if variaveis_musica >= 64:
    print('A música tem ALTAS chances de ser POPULAR')
else: 
    print('A música tem BAIXAS chances de ser POPULAR')

A música tem BAIXAS chances de ser POPULAR


In [19]:
dot_data = tree.export_graphviz(arvore, out_file=None, 
                                feature_names=X_train.columns,  
                                class_names=tophits_df['popularity'] ,
                                filled=True)

graph = graphviz.Source(dot_data, format="png") 
graph

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH